# CS-GY 6313/CUSP-GX 6006 Information Assignment 1

Given: Two triangle positions in 3D, triangle colors, and camera parameters. 

Output: Saved and visualizable color images from the three given cameras (as .png, .jpg, etc.)

##Inputs:

3D Coordinates of the vertices two triangles (vertices1, vertices2, vertices3)

```
tri1_vertices = [(x0, y0, z0), (x1, y1, z1), (x2, y2, z2)]
tri2_vertices = [(x0, y0, z0), (x1, y1, z1), (x2, y2, z2)] 
```
Camera parameters (position, forward/facing direction, up/camera orientation, fov). Assume aspect ratio is 1:1 and FOV in degrees.
```
cam1 = [(x0, y0, z0), (x0_f, y0_f, z0_f), (x0_u, y0_u, z0_u), fov0]
cam2 = [(x1, y1, z1), (x1_f, y1_f, z1_f), (x1_u, y1_u, z1_u), fov1]
cam3 = [(x2, y2, z2), (x2_f, y2_f, z2_f), (x2_u, y2_u, z2_u), fov2]
```



Changelog: 
10/6: Fixed last row of translate_center_to_origin to be [0, 0, 0, 1] instead of [0, 0, 1, 0]
9/29: Fixed a variable typo in scale_by_2_matrix to 2/(t-b), orginally 2/(l-b) 

# Programming exercises

In [10]:
import matplotlib.pyplot as plt 
import numpy as np

## Render triangles in camera using CPU-only rasterisation.


Write a function to render an image using camera settings and triangle vertices as the input. Write your own code for rasterization to render a triangle pixel-by-pixel. You can use any method to show the triangles (ex. Matplotlib, OpenCV, plotly, etc.) as long as you first perform Model-View-Projection.






# Complete the functions for Model View Projection

Model: objects to coordinates in world space

View: from world space to camera space

Projection: from camera space to clip/screen space

In [11]:
# Helper function
def normalize(array):
  return array / np.linalg.norm(array)

def area(x1, y1, x2, y2, x3, y3):
  return abs((x1 * (y2 - y3) + x2 * (y3 - y1)
    + x3 * (y1 - y2)) / 2.0)
  
def isInside(tri, x, y):
  A = area (tri[0][0], tri[0][1], tri[1][0], tri[1][1], tri[2][0], tri[2][1])
  A1 = area (x, y, tri[1][0], tri[1][1], tri[2][0], tri[2][1])
  A2 = area (tri[0][0], tri[0][1], x, y, tri[2][0], tri[2][1])
  A3 = area (tri[0][0], tri[0][1], tri[1][0], tri[1][1], x, y)
  if(round(A, 4) == round(A1 + A2 + A3, 4)):
    return True
  else:
    return False

# ax + by + cz = d
def getPlane(tri):
  v1 = tri[1]-tri[0]
  v2 = tri[2]-tri[1]
  cp = np.cross(v1, v2)
  a, b, c = cp
  d = np.dot(cp, tri[2])
  return np.array([a,b,c,d])

In [12]:
def view(camera, triangle1, triangle2):
  # get R_view
  D = np.subtract(camera[0],camera[1])
  D = normalize(D)
  U = normalize(camera[2])
  R = np.cross(U,D)
  R_view = [[R[0],  R[1], R[2], 0],
            [U[0],  U[1], U[2], 0],
            [-D[0],-D[1],-D[2], 0],
            [0,     0,    0,    1]]
  print('R_view', R_view)
  # get T_view
  T_view = np.block([[np.eye(3),-np.array([camera[0]]).T],
                     [np.zeros((1,3)), 1]])
  print('T_view', T_view)
  # get M_view
  M_view = np.matmul(R_view, T_view)
  print('M_view', M_view)

  # get triangle in camera space
  tri1_in_camera_space = []
  for p in triangle1:
    p_t = np.append(p,1)
    newP = np.matmul(M_view, p_t.T)
    newP = newP[0:3]
    tri1_in_camera_space.append(newP)
  tri2_in_camera_space = []
  for p in triangle2:
    p_t = np.append(p,1)
    newP = np.matmul(M_view, p_t.T)
    newP = newP[0:3]
    tri2_in_camera_space.append(newP)

  return  np.array(tri1_in_camera_space), np.array(tri2_in_camera_space), M_view

In [13]:
def get_M_persp(field_of_view):
  aspect_ratio = 1
  n = 0.01
  f = 1000
  r = n*np.tan(field_of_view/2*np.pi/180)
  l = -r
  t = aspect_ratio*r
  b = -t
  
  scale_by_2 = [[2/(r-l), 0, 0, 0], 
               [0, 2/(t-b), 0 , 0],
               [0, 0, 2/(n-f), 0],
               [0, 0, 0, 1]]

  translate_center_to_origin = [[1, 0, 0, -(r+l)/2],
                                [0, 1, 0, -(t+b)/2],
                                [0, 0, 1, -(n+f)/2],
                                [0, 0, 0, 1]]

  M_ortho = np.matmul(scale_by_2, translate_center_to_origin)
  
  M_persp = np.matmul(M_ortho,
             [[n, 0, 0, 0],
             [0, n, 0, 0],
             [0, 0, n+f, -n*f],
             [0, 0, 1, 0]])

  M_persp = np.array([[1/(aspect_ratio*np.tan(field_of_view/2*np.pi/180)), 0, 0 ,0],
            [0, 1/np.tan(field_of_view/2*np.pi/180), 0 ,0],
            [0, 0, (n+f)/(n-f), (-2*n*f)/(n-f)],
            [0, 0, 1, 0]])
  
  return M_persp

def perspective_projection(field_of_view, tri1, tri2):
  M_persp = get_M_persp(field_of_view)
  
  tri_1_screen_coordinates=[]
  for p in tri1:
    p_t = np.append(p,1)
    newP = np.matmul(M_persp, p_t.T)
    newP = newP[0:3]/newP[3]
    tri_1_screen_coordinates.append(newP)

  tri_2_screen_coordinates=[]
  for p in tri2:
    p_t = np.append([p],1)
    newP = np.matmul(M_persp, p_t.T)
    newP = newP[0:3]/newP[3]
    tri_2_screen_coordinates.append(newP)
  
  return np.array(tri_1_screen_coordinates), np.array(tri_2_screen_coordinates), M_persp

In [14]:
def mvp(camera, triangle1, triangle2):
  tri1_in_camera_space, tri2_in_camera_space, M_view = view(camera, triangle1, triangle2)
  print("M_view\n:", M_view)
  print("Triangle1 in camera space:\n", tri1_in_camera_space)
  print("Triangle1 in camera space:\n", tri2_in_camera_space)
  tri1_in_screen_space, tri2_in_screen_space, M_persp = perspective_projection(camera[3], tri1_in_camera_space, tri2_in_camera_space)
  print("M_persp\n:", M_persp)
  print("Triangle1 in screen space:\n", tri1_in_screen_space)
  print("Triangle2 in screen space:\n", tri2_in_screen_space)
  return tri1_in_screen_space, tri2_in_screen_space

In [15]:
def rasterize(tri1, tri2):
  pixels = np.zeros(shape = (100, 100, 3)) # 100x100 matrix of RGB values
  for row in range(100):
    for col in range(100):
      _row = round(row/50-1,2)
      _col = round(col/50-1,2)
      if isInside(tri1, _col, _row):
        pixels[row][col] = [255, 0, 0]
      elif isInside(tri2, _col, _row):
        pixels[row][col] = [0, 255, 0]
      else:
        pixels[row][col] = [0, 0, 0]
  return pixels

In [16]:
def full_rasterize(tri1, tri2):
  tri1, tri2 = (tri1+1)*50, (tri2+1)*50
  pixels = np.zeros(shape = (100, 100, 3)) # 100x100 matrix of RGB values 
  superSampling=[[-0.25, -0.25],[-0.25, 0.25],[0.25,-0.25],[0.25, 0.25]]
  sampleLen = len(superSampling)
  # intersectionCheck = [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
  rplane=getPlane(tri1)
  gplane=getPlane(tri2)
  
  for row in range(100):
    for col in range(100):
      rcount, gcount = 0, 0
      # anti-aliasing 
      for s in superSampling:
        if isInside(tri1, col+s[1], row+s[0]): # Reverse row and col
          rcount += 1
        if isInside(tri2, col+s[1], row+s[0]):
          gcount += 1
      pixels[row][col] = [255*rcount/sampleLen, 255*gcount/sampleLen, 0]
      # occlusion
      rz = (rplane[3] - rplane[0]*col - rplane[1]*row)/rplane[2]
      gz = (gplane[3] - gplane[0]*col - gplane[1]*row)/gplane[2]
      if rz > gz and rcount == sampleLen:
        pixels[row][col] = [255, 0, 0]
      if rz < gz and gcount == sampleLen:
        pixels[row][col] = [0, 255, 0]
  # # intersection anti-aliasing
  # for row in range(1,99):
  #   for col in range(1,99):
  #     rcount, gcount = 0, 0
  #     for p in intersectionCheck:
  #       if (pixels[row+p[0]][col+p[1]]==[255,0,0]).all():
  #         rcount+=1
  #       if (pixels[row+p[0]][col+p[1]]==[0,255,0]).all():
  #         gcount+=1
  #     if gcount>2 and rcount>1:
  #       pixels[row][col]=[255*rcount/sampleLen, 255*gcount/sampleLen, 0]
  return pixels

In [17]:
# Triangle vertices in 3D world space
tri1 = [(0, 0, 0), (0, 30, 0), (35, 0, 35)] # RGB(255, 0, 0), Red
tri2 = [(17, 0, 0), (0, 0, 17), (17, 45, 17)] # RGB(0, 255, 0), Green

# cam = [(x, y, z), (x_f, y_f, z_f), (x_u, y_u, z_u), fov]
# The first coordinate is where the camera is in world space. The second is what
# coordinate it is looking at/facing. The third represents the "up" axis.  
# Ex. cam1, cam3 up = positive of y-axis. cam2 up = negative if y-axis (upside-down camera)
cam1 = [(50, 10, 0), (0, 10, 0), (0, 1, 0), 90] 
cam2 = [(50, 10, 0), (0, 10, 0), (0, -1, 0), 105]
cam3 = [(0, 10, 60), (0, 10, 0), (0, 1, 0), 120] 

perspective1_tri1, perspective1_tri2 = mvp(cam1, tri1, tri2)
# perspective2_tri1, perspective2_tri2 = mvp(cam2, tri1, tri2)
# perspective3_tri1, perspective3_tri2 = mvp(cam3, tri1, tri2)

# # Draw triangles in 2D
# perspective1 = full_rasterize(perspective1_tri1, perspective1_tri2) 
# perspective2 = full_rasterize(perspective2_tri1, perspective2_tri2)
# perspective3 = full_rasterize(perspective3_tri1, perspective3_tri2)

R_view [[0.0, 0.0, -1.0, 0], [0.0, 1.0, 0.0, 0], [-1.0, -0.0, -0.0, 0], [0, 0, 0, 1]]
T_view [[  1.   0.   0. -50.]
 [  0.   1.   0. -10.]
 [  0.   0.   1.   0.]
 [  0.   0.   0.   1.]]
M_view [[  0.   0.  -1.   0.]
 [  0.   1.   0. -10.]
 [ -1.   0.   0.  50.]
 [  0.   0.   0.   1.]]
M_view
: [[  0.   0.  -1.   0.]
 [  0.   1.   0. -10.]
 [ -1.   0.   0.  50.]
 [  0.   0.   0.   1.]]
Triangle1 in camera space:
 [[  0. -10.  50.]
 [  0.  20.  50.]
 [-35. -10.  15.]]
Triangle1 in camera space:
 [[  0. -10.  33.]
 [-17. -10.  50.]
 [-17.  35.  33.]]
M_persp
: [[ 1.         0.         0.         0.       ]
 [ 0.         1.         0.         0.       ]
 [ 0.         0.        -1.00002    0.0200002]
 [ 0.         0.         1.         0.       ]]
Triangle1 in screen space:
 [[ 0.         -0.2        -0.99962   ]
 [ 0.          0.4        -0.99962   ]
 [-2.33333333 -0.66666667 -0.99868665]]
Triangle2 in screen space:
 [[ 0.         -0.3030303  -0.99941393]
 [-0.34       -0.2        -0.99962


Save the 100x100x3 RGB images you rendered with the given camera settings and triangle vertices. Images can be displayed using MatplotLib.

In [18]:
# Save and view images
# origin = "lower" sets origin of plot to bottom left
plt.imshow(perspective1.astype(np.uint8), origin="lower") 
plt.show()
 
plt.imshow(perspective2.astype(np.uint8), origin="lower") 
plt.show()

plt.imshow(perspective3.astype(np.uint8), origin="lower") 
plt.show()

NameError: name 'perspective1' is not defined

# Submission details

Click "File" > "Save a Copy in Drive" and rename the file to your netID followed by "_InfoVisHW1." Ex. "bsl334_InfoVisHW1.ipynb"

You will be submitting your Colab notebook along with your three camera perspective images. 



Grading

```
Single triangle with MVP (50%)
Single triangle with MVP and anti-aliasing (75%)
Two triangles with MVP, anti-aliasing, and occlusion (100%) 
```